In [22]:
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import numpy as np

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/cameronfaulkner/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cameronfaulkner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset = pd.read_csv("/Users/cameronfaulkner/Documents/Cogs 118a/Final Project/Sentiment Added Dataset")
dataset.head()

,rawContent,replyCount,retweetCount,likeCount,quoteCount,links,media,mentionedUsers,hashtags,Party,Positive,Negative,Neutral
0,New Year’s Day in Seward overlooking the harbo...,44.0,12.0,239.0,2.0,0,3,0,0,1,1,0,0
1,Happy #NewYear!,19.0,4.0,22.0,1.0,0,0,0,1,1,1,0,0
2,I championed provisions to strengthen Alaska’s...,9.0,5.0,19.0,0.0,0,0,0,0,1,1,0,0
3,"December: As senior Appropriator, I helped wri...",11.0,6.0,28.0,3.0,0,1,0,0,1,1,0,0
4,December: Congressman Don Young’s legacy will ...,5.0,9.0,46.0,1.0,0,1,0,0,1,1,0,0


In [46]:
dataset.shape[0]

786268

In [8]:
dataset["rawContent"].iloc[0:10]

0    new year day seward overlooking harbor beautif...
1                                        happy newyear
2    championed provision strengthen alaska economy...
3    december senior appropriator helped write majo...
4    december congressman young legacy live many wa...
5    november long supported marriage equality beli...
6    october great anchorage many alaskan nativefed...
7    september senatordansullivan rep peltola govdu...
8    september fall typhoon merbok produced widespr...
9    august visit air station sitka comdtuscg uscg ...
Name: rawContent, dtype: object

In [41]:
def text_preprocess(ds: pd.Series) -> pd.Series:
    """
    Apply NLP Preprocessing Techniques to the reviews.
    """
    for m in range(len(ds)):
        
        main_words = re.sub('[^a-zA-Z]', ' ', ds[m])                                      # Retain only alphabets
        main_words = (main_words.lower()).split()
        main_words = [w for w in main_words if not w in set(stopwords.words('english'))]  # Remove stopwords
        
        lem = WordNetLemmatizer()
        main_words = [lem.lemmatize(w) for w in main_words if len(w) > 1]                 # Group different forms of the same word
        
        main_words = ' '.join(main_words)
        ds[m] = main_words
    return ds

test_processed = text_preprocess(dataset['rawContent'].iloc[0:100000])

In [14]:
#X = dataset.iloc[:, 0].values
X = test_processed.values
#y = dataset.iloc[:, 1].values

# Building a TF IDF matrix out of the corpus of reviews
from sklearn.feature_extraction.text import TfidfVectorizer
td = TfidfVectorizer(max_features = 4500)
X = td.fit_transform(X).toarray()
print(X.shape)

(10000, 4500)


In [16]:
dataset.columns

Index(['rawContent', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'links', 'media', 'mentionedUsers', 'hashtags', 'Party', 'Positive',
       'Negative', 'Neutral'],
      dtype='object')

In [42]:
sub_dataset = dataset[['replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'links', 'media', 'mentionedUsers', 'hashtags', 'Positive',
       'Negative', 'Neutral']]
sub_dataset = sub_dataset.iloc[0:100000]

In [43]:
np.shape(sub_dataset.to_numpy())

(100000, 11)

In [45]:
#X = dataset.iloc[:, 0].values
X = test_processed.values
#y = dataset.iloc[:, 1].values
y = dataset["Party"][0:100000].values

# Building a TF IDF matrix out of the corpus of reviews
from sklearn.feature_extraction.text import TfidfVectorizer
td = TfidfVectorizer(max_features = 4500)
X = td.fit_transform(X).toarray()
print(np.shape(X))
X = np.append(X,sub_dataset.to_numpy(),axis=1)
print(np.shape(X))
# Splitting into training & test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,
                                                    random_state = 0)

# Training the classifier & predicting on test data
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

# Classification metrics
from sklearn.metrics import accuracy_score, classification_report
classification_report = classification_report(y_test, y_pred)

print('\n Accuracy: ', accuracy_score(y_test, y_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)

(100000, 4500)
(100000, 4511)

 Accuracy:  0.7823333333333333

Classification Report

               precision    recall  f1-score   support

           0       0.89      0.66      0.76     15466
           1       0.72      0.91      0.80     14534

    accuracy                           0.78     30000
   macro avg       0.80      0.79      0.78     30000
weighted avg       0.81      0.78      0.78     30000

